In [ ]:
import geopandas as gpd
import os
from datetime import date

SHARED_PROJECT_PATH = '...'
SHARED_PROJECT_PATH_POLY = '...'

In [ ]:
# Read land use data
gdf_land_use = gpd.read_file(os.path.join(SHARED_PROJECT_PATH, 'data', 'land_use', 'KSH', 'agg', 'BP_land-use_agg.geojson')).drop(columns=['comment']).dropna()

In [ ]:
# Project land use data to EOV
gdf_land_use = gdf_land_use.to_crs('EPSG:23700')

In [ ]:
# Process and aggregate land use data
others_list = ['Isolated structures', 'Wetlands', 'Land without current use']
gdf_land_use.loc[list(gdf_land_use[gdf_land_use['class_2018'].isin(others_list)].index), 'class_2018'] = 'Other'
private_list = ['Port areas', 'Construction sites', 'Airports', 'Industrial, commercial, public, military and private units', 'Mineral extraction and dump sites', 'Railways and associated land']
gdf_land_use.loc[list(gdf_land_use[gdf_land_use['class_2018'].isin(private_list)].index), 'class_2018'] = 'lu_com_ind'
urban_list = ['Green urban areas', 'Sports and leisure facilities', 'Medium-low density urban fabric (S.L. : < 50%)', 'Urban fabric (S.L. : > 50%)']
gdf_land_use.loc[list(gdf_land_use[gdf_land_use['class_2018'].isin(urban_list)].index), 'class_2018'] = 'lu_urban'
green_list = ['Crops, pastures', 'Forests']
gdf_land_use.loc[list(gdf_land_use[gdf_land_use['class_2018'].isin(green_list)].index), 'class_2018'] = 'lu_green'
gdf_land_use.drop(gdf_land_use[gdf_land_use['class_2018'].isin(['Other', 'Roads and associated land', 'Water'])].index, inplace=True)
gdf_land_use = gdf_land_use.dissolve(by='class_2018', aggfunc='first')[['geometry']].reset_index()

In [ ]:
# Read processed OSM raod network data
today = date.today().strftime('%d-%m-%Y')
gdf_graph_edges = gpd.read_file(os.path.join(SHARED_PROJECT_PATH, 'data', 'OSM_road_network', today, f'BP_safety-network_{today}.json'), driver='GeoJSON')

In [ ]:
# Add land use data to OSM road network as percentage of land use in road segment buffer
gdf_graph_edges_buff = gdf_graph_edges.copy()
gdf_graph_edges_buff.geometry = gdf_graph_edges_buff.geometry.buffer(25, cap_style=2)
gdf_res = gpd.overlay(gdf_graph_edges_buff, gdf_land_use, how='intersection')
gdf_res['land_use_perc'] = gdf_res.geometry.area
gdf_res['land_use_perc'] /= gdf_res.join(gdf_res.groupby('gnx_edge_id').land_use_perc.sum(), on='gnx_edge_id', rsuffix='_sum').land_use_perc_sum
gdf_res = gdf_res.pivot_table(index=['gnx_edge_id'], columns=['class_2018'], values='land_use_perc').fillna(0)
gdf_res.columns = [col for col in gdf_res.columns]
gdf_res.reset_index(inplace=True)
gdf_graph_edges = gdf_graph_edges.merge(gdf_res, on='gnx_edge_id')

In [ ]:
# Save road safety network with land use percentages as geojson
gdf_graph_edges.to_file(os.path.join(SHARED_PROJECT_PATH, 'data', 'OSM_road_network', today, f'BP_safety-network_{today}.json'), driver='GeoJSON')